In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install pygraphviz

In [3]:
from incremental_parsing.lex_earley.middle_earley import create_middle_bnf, create_parse_hierarchy, \
    create_bnf_direct
from incremental_parsing.lex_earley.lark_grammar import get_python_context
from incremental_parsing.lex_earley.lex_earley import lex_earley_init, lex_earley_run, _to_suffix_parser_state, force_eof
from incremental_parsing.lex_earley.earley_nfa import EarleyNFA, tokens_to_nfa
from incremental_parsing.evaluation.text_cuts import cut_text_random
import networkx as nx
import datasets
import random

In [4]:
context = get_python_context()
init_state = lex_earley_init(context)
dataset = datasets.load_dataset("bigcode/the-stack-smol-xl", data_dir="data/python")["train"]

Found cached dataset json (/Users/dmelcer/.cache/huggingface/datasets/bigcode___json/bigcode--the-stack-smol-xl-1fe14832da3eae85/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Load data from the stack
data = dataset[6751]["content"]
random.seed(hash((6751, 5)) % (2 ** 32))
prefix, _, suffix = cut_text_random(data, 0, .9, .2)
suffix += "\n"  # This line is important

In [6]:
# Or manually specify a prefix and suffix
prefix = ""
suffix = "else d\n"
pre_branch_num = 0
post_branch_num = 0

In [7]:
pre = lex_earley_run(context=context, state=init_state, value=prefix)
g_pre = create_parse_hierarchy(context.grammar, pre.branches[pre_branch_num].parser_state.earley_trie.charts, (len(pre.branches[pre_branch_num].parser_state.earley_trie.charts) - 1,))
ag = nx.nx_agraph.to_agraph(g_pre.reverse())  # Formalization assumes arrows go in one direction, implementation has arrows go in other direction
ag.layout(prog="dot")
ag.draw("pre.png")
suf = _to_suffix_parser_state(context=context, state=pre, suffix=suffix, make_dummy_trie=True)
post = lex_earley_run(context=context, state=suf, value=suffix)
post_eof = force_eof(context, post)
branch = post_eof.branches[post_branch_num]
tokens_reverse = list(branch.suffix_state.parser_state.earley_trie.get_reverse_token_sequence())
token_nfa, final_states = tokens_to_nfa(tokens_reverse)
earley_nfa = EarleyNFA(context.grammar.reverse(), token_nfa)
g_post = create_parse_hierarchy(context.grammar, earley_nfa.charts, final_states, reverse_state_positions=True)
ag = nx.nx_agraph.to_agraph(g_post.reverse())
ag.layout(prog="dot")
ag.draw("post.png")


In [8]:
r = create_middle_bnf(context.grammar, g_pre, g_post, (len(pre.branches[pre_branch_num].parser_state.earley_trie.charts) - 1,), final_states)
print(str(r))
print(r.top_level_rules)

decorator : AT dotted_name LPAR arguments RPAR _NEWLINE
 | AT dotted_name LPAR RPAR _NEWLINE
 | AT dotted_name _NEWLINE

decorators : __decorators_plus_2

decorated : decorators classdef
 | decorators funcdef
 | decorators async_funcdef

async_funcdef : ASYNC funcdef

funcdef : DEF name LPAR parameters RPAR __ANON_0 test COLON suite
 | DEF name LPAR parameters RPAR COLON suite
 | DEF name LPAR RPAR __ANON_0 test COLON suite
 | DEF name LPAR RPAR COLON suite

parameters : paramvalue __parameters_star_3 COMMA SLASH __parameters_star_3 COMMA starparams
 | paramvalue __parameters_star_3 COMMA SLASH __parameters_star_3 COMMA kwparams
 | paramvalue __parameters_star_3 COMMA SLASH __parameters_star_3 COMMA
 | paramvalue __parameters_star_3 COMMA SLASH __parameters_star_3
 | paramvalue __parameters_star_3 COMMA SLASH COMMA starparams
 | paramvalue __parameters_star_3 COMMA SLASH COMMA kwparams
 | paramvalue __parameters_star_3 COMMA SLASH COMMA
 | paramvalue __parameters_star_3 COMMA SLASH
 | 

In [9]:
r = create_bnf_direct(grammar=context.grammar.reverse(), final_chart_indices=final_states, charts=earley_nfa.charts, is_right_context=True).reverse()
print(str(r))
print(r.top_level_rules)

decorator : AT dotted_name LPAR arguments RPAR _NEWLINE
 | AT dotted_name LPAR RPAR _NEWLINE
 | AT dotted_name _NEWLINE

decorators : __decorators_plus_2

decorated : decorators classdef
 | decorators funcdef
 | decorators async_funcdef

async_funcdef : ASYNC funcdef

funcdef : DEF name LPAR parameters RPAR __ANON_0 test COLON suite
 | DEF name LPAR parameters RPAR COLON suite
 | DEF name LPAR RPAR __ANON_0 test COLON suite
 | DEF name LPAR RPAR COLON suite

parameters : paramvalue __parameters_star_3 COMMA SLASH __parameters_star_3 COMMA starparams
 | paramvalue __parameters_star_3 COMMA SLASH __parameters_star_3 COMMA kwparams
 | paramvalue __parameters_star_3 COMMA SLASH __parameters_star_3 COMMA
 | paramvalue __parameters_star_3 COMMA SLASH __parameters_star_3
 | paramvalue __parameters_star_3 COMMA SLASH COMMA starparams
 | paramvalue __parameters_star_3 COMMA SLASH COMMA kwparams
 | paramvalue __parameters_star_3 COMMA SLASH COMMA
 | paramvalue __parameters_star_3 COMMA SLASH
 | 